Инициализация

In [46]:
import numpy as np
import tensorflow as tf
import spektral

Cora - набор данных для обучения представлению графов (потом надо найти инфу по ней из видоса)
spektral позволяет нам быстро загружать и предварительно обрабатывать многие стандартные наботы данных для обучения представлению графов (к примеру, Cora). Она поставляется с полезными функциями загрузки, которые позволят нам прямой доступ к таким элементам, как:
adj - матрица смежности графа
features - матрица функций, которая дает нам функцию в каждом из узлов
labels - метки, обозначающие тему каждой статьи
train_mask - массив масок: какие узлы принадлежат обучающему набору (training set)
val_mask - массив масок: какие узлы принадлежат проверочному набору (validation set)
test_mask - массив масок: какие узлы принадлежат тестовому набору (test set)

Загружаем эти данные из набора данных 'Cora'

In [47]:
cora_dataset = spektral.datasets.citation.Citation(name='Cora')
test_mask = cora_dataset.mask_te
train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
graph = cora_dataset.graphs[0]
features = graph.x
adj = graph.a
labels = graph.y

#features = features.todense()
#adj = adj + np.eye(adj.shape[0])
features = features.astype('float32')
adj = adj.astype('float32')

#Выведем размеры набора данных Cora
print(graph)
print(features.shape)
print(adj.shape)
print(labels.shape)

#Выведем количество узлов в каждом наборе
print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)
(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


Опредем функцию потери перекрестной энтропии и посчета точности вычисления для конкретного набора (mask)

In [48]:
def masked_softmax_cross_entropy(logits, labels, mask):
    loss = tf.nn.softmax_cross_entropy_with_logits(logist=logits, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)

def masked_accuracy(logits, labels, mask):
    corrent_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(corrent_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

Определим слой GNN
fts - матрица признаков узлов
adj - матрица смежности
transform - некоторая трансформация, которая будет применяться к каждому узлу
activation - функция активации

In [49]:
def gnn(fts, adj, transform, activation):
    seq_fts = transform(fts) #эквивалент матрицы весов W
    ret_fts = tf.matmul(adj, seq_fts)
    return activation(ret_fts)

Определим двуслойную GNN для обучения на данных Cora
fts - матрица признаков узлов
adj - матрица смежности
gnn_fn - некоторая модель GNN 
units - количество единиц, которые будет вычислять gnn в каждом узле (сколько изменений в скрытых функциях)
epochs - количество эпох обучения
lr - скорость обучения


In [50]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
    layer_1 = tf.keras.layers.Dense(units) #скрытый слой равный числу units
    layer_2 = tf.keras.layers.Dense(7) #определяет классификацию каждого узла

    def cora_gnn(fts, adj):
        hidden = gnn_fn(fts, adj, layer_1, tf.nn.relu) #tf.nn.relu - нелинейная функция активации
        logits = gnn_fn(hidden, adj, layer_2, tf.identity)
        return logits
    
    optimazer = tf.keras.optimizers.Adam(learning_rate=lr)

    best_accuracy = 0.0
    for ep in range(epochs + 1):
        with tf.GradientTape() as t:
            print("debug: ", fts, adj)
            logits = cora_gnn(fts, adj)
            loss = masked_softmax_cross_entropy(logits, labels, train_mask)
        
        variables = t.watched_variables()
        grads = t.gradient(loss, variables)
        optimazer.apply_gradients(zip(grads, variables))

        logits = cora_gnn(fts, adj)
        val_accuracy = masked_accuracy(logits, labels, val_mask)
        test_accuracy = masked_accuracy(logits, labels, test_mask)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            print('Epoch', ep, '| Training loss:', loss.numpy(), '| Val accuracy:', val_accuracy.numpy(), '| Test accuracy:', test_accuracy.numpy())

Главный вызов обучения GNN

In [51]:
train_cora(features, adj, gnn, 32, 200, 0.01)

debug:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]   (0, 633)	1.0
  (0, 1862)	1.0
  (0, 2582)	1.0
  (1, 2)	1.0
  (1, 652)	1.0
  (1, 654)	1.0
  (2, 1)	1.0
  (2, 332)	1.0
  (2, 1454)	1.0
  (2, 1666)	1.0
  (2, 1986)	1.0
  (3, 2544)	1.0
  (4, 1016)	1.0
  (4, 1256)	1.0
  (4, 1761)	1.0
  (4, 2175)	1.0
  (4, 2176)	1.0
  (5, 1629)	1.0
  (5, 1659)	1.0
  (5, 2546)	1.0
  (6, 373)	1.0
  (6, 1042)	1.0
  (6, 1416)	1.0
  (6, 1602)	1.0
  (7, 208)	1.0
  :	:
  (2694, 431)	1.0
  (2694, 2695)	1.0
  (2695, 431)	1.0
  (2695, 2694)	1.0
  (2696, 2615)	1.0
  (2697, 986)	1.0
  (2698, 1400)	1.0
  (2698, 1573)	1.0
  (2699, 2630)	1.0
  (2700, 1151)	1.0
  (2701, 44)	1.0
  (2701, 2624)	1.0
  (2702, 186)	1.0
  (2702, 1536)	1.0
  (2703, 1298)	1.0
  (2704, 641)	1.0
  (2705, 287)	1.0
  (2706, 165)	1.0
  (2706, 169)	1.0
  (2706, 1473)	1.0
  (2706, 2707)	1.0
  (2707, 165)	1.0
  (2707, 598)	1.0
  (2707, 1473)	1.

ValueError: TypeError: sparse array length is ambiguous; use getnnz() or shape[0]
Traceback (most recent call last):

  File "c:\Users\leshk\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\sparse\_base.py", line 404, in __len__
    raise TypeError("sparse array length is ambiguous; use getnnz()"

TypeError: sparse array length is ambiguous; use getnnz() or shape[0]

